## Scenario 2: A cross-functional team with one data scientist working on an ML model


MLflow setup:
- tracking server: yes, local server
- backend store: sqlite database
- artifacts store: local filesystem

The experiments can be explored locally by accessing the local tracking server.

To run this example you need to launch the mlflow server locally by running the following command in your terminal:

`mlflow server --backend-store-uri sqlite:///backend.db`

In [2]:
! mlflow server --backend-store-uri sqlite:///backend.db

Option 'default-artifact-root' is required when backend store is not local file based.
Usage: mlflow server [OPTIONS]
Try 'mlflow server --help' for help.

Error: Option 'default-artifact-root' is required when backend store is not local file based.


One needs to define a `default-artifact-root` to run `mlflow server`

In [3]:
! mlflow server --backend-store-uri sqlite:///backend.db --default-artifact-root ./artifacts_local

2022/06/01 16:20:55 INFO mlflow.store.db.utils: Creating initial MLflow database tables...
2022/06/01 16:20:55 INFO mlflow.store.db.utils: Updating database tables
INFO  [alembic.runtime.migration] Context impl SQLiteImpl.
INFO  [alembic.runtime.migration] Will assume non-transactional DDL.
INFO  [alembic.runtime.migration] Running upgrade  -> 451aebb31d03, add metric step
INFO  [alembic.runtime.migration] Running upgrade 451aebb31d03 -> 90e64c465722, migrate user column to tags
INFO  [alembic.runtime.migration] Running upgrade 90e64c465722 -> 181f10493468, allow nulls for metric values
INFO  [alembic.runtime.migration] Running upgrade 181f10493468 -> df50e92ffc5e, Add Experiment Tags Table
INFO  [alembic.runtime.migration] Running upgrade df50e92ffc5e -> 7ac759974ad8, Update run tags with larger limit
INFO  [alembic.runtime.migration] Running upgrade 7ac759974ad8 -> 89d4b8295536, create latest metrics table
INFO  [89d4b8295536_create_latest_metrics_table_py] Migration complete!
INFO  

And now, as you can see the server was created!

In [4]:
import mlflow


mlflow.set_tracking_uri("http://127.0.0.1:5000")

In [5]:
print(f"tracking URI: '{mlflow.get_tracking_uri()}'")

tracking URI: 'http://127.0.0.1:5000'


In [6]:
mlflow.list_experiments()

[<Experiment: artifact_location='./artifacts_local/0', experiment_id='0', lifecycle_stage='active', name='Default', tags={}>]

In [7]:
from sklearn.linear_model import LogisticRegression
from sklearn.datasets import load_iris
from sklearn.metrics import accuracy_score

mlflow.set_experiment("my-experiment-1")

with mlflow.start_run():

    X, y = load_iris(return_X_y=True)

    params = {"C": 0.1, "random_state": 42}
    mlflow.log_params(params)

    lr = LogisticRegression(**params).fit(X, y)
    y_pred = lr.predict(X)
    mlflow.log_metric("accuracy", accuracy_score(y, y_pred))

    mlflow.sklearn.log_model(lr, artifact_path="models")
    print(f"default artifacts URI: '{mlflow.get_artifact_uri()}'")

2022/06/01 16:23:51 INFO mlflow.tracking.fluent: Experiment with name 'my-experiment-1' does not exist. Creating a new experiment.


default artifacts URI: './artifacts_local/1/e52e8c747d4e47cb9e93a6606744a280/artifacts'


In [8]:
mlflow.list_experiments()

[<Experiment: artifact_location='./artifacts_local/0', experiment_id='0', lifecycle_stage='active', name='Default', tags={}>,
 <Experiment: artifact_location='./artifacts_local/1', experiment_id='1', lifecycle_stage='active', name='my-experiment-1', tags={}>]

### Interacting with the model registry

In [9]:
from mlflow.tracking import MlflowClient


client = MlflowClient("http://127.0.0.1:5000")

In [10]:
client.list_registered_models()

[]

In [11]:
run_id = client.list_run_infos(experiment_id='1')[0].run_id
mlflow.register_model(
    model_uri=f"runs:/{run_id}/models",
    name='iris-classifier'
)

Successfully registered model 'iris-classifier'.
2022/06/01 16:25:23 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation.                     Model name: iris-classifier, version 1
Created version '1' of model 'iris-classifier'.


<ModelVersion: creation_timestamp=1654097123411, current_stage='None', description='', last_updated_timestamp=1654097123411, name='iris-classifier', run_id='e52e8c747d4e47cb9e93a6606744a280', run_link='', source='./artifacts_local/1/e52e8c747d4e47cb9e93a6606744a280/artifacts/models', status='READY', status_message='', tags={}, user_id='', version='1'>

In [12]:
client.list_registered_models()

[<RegisteredModel: creation_timestamp=1654097123383, description='', last_updated_timestamp=1654097123411, latest_versions=[<ModelVersion: creation_timestamp=1654097123411, current_stage='None', description='', last_updated_timestamp=1654097123411, name='iris-classifier', run_id='e52e8c747d4e47cb9e93a6606744a280', run_link='', source='./artifacts_local/1/e52e8c747d4e47cb9e93a6606744a280/artifacts/models', status='READY', status_message='', tags={}, user_id='', version='1'>], name='iris-classifier', tags={}>]